In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import liana as li
import anndata
import scanpy as sc
from openproblems import tasks
import os 

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from utils import *

1.13.1+cu117
1.13.1+cu117


In [ ]:
import argparse
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Entities
from torch_geometric.nn import FastRGCNConv, RGCNConv, GCNConv, InnerProductDecoder, GAE, VGAE
from torch_geometric.utils import k_hop_subgraph

from torch_geometric.datasets import Planetoid
from torch_geometric.data.data import Data
from torch_geometric.transforms import NormalizeFeatures


from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity

from scipy.io import mmread

# Pre-processing from original matrix


In [ ]:
adata = sc.read_h5ad("/data/Sikander_data/Visium-FZ_GT_P19.h5ad")

In [ ]:
matrix = pd.DataFrame.sparse.from_spmatrix(adata.X,index=adata.obs.index.tolist(),columns=adata.var["feature_name"].tolist())

In [ ]:
meta = pd.DataFrame({"cell":adata.obs.index.tolist(),"labels":adata.obs["cell_type_original"].tolist()})
meta.index = meta["cell"].tolist()

In [ ]:
matrix = matrix.transpose()

In [ ]:
matrix = matrix[meta.index.tolist()]

In [ ]:
from scipy.stats import wilcoxon
import anndata
import scanpy as sc

In [ ]:
index = matrix.index.tolist()

In [ ]:
meta['labels'] = meta["labels"].astype("string").tolist()

In [ ]:
meta["cell"] = meta["cell"].astype("category")
meta["labels"] = meta["labels"].astype("category")

In [ ]:
matrix = matrix.loc[:,~matrix.columns.duplicated()].copy()
meta = meta.loc[~meta.index.duplicated(),:].copy()
cell_group = meta.groupby("labels").count()
valid_cell_groups = cell_group[cell_group["cell"] > 1].index.tolist()
meta = meta[meta["labels"].isin(valid_cell_groups)]
meta["labels"] = meta["labels"].cat.remove_unused_categories()
matrix = matrix[meta["cell"].tolist()]



In [ ]:
adata = anndata.AnnData(matrix.transpose())

In [ ]:
sc.pp.filter_genes(adata, min_cells=10)


In [ ]:
#adata = anndata.AnnData(matrix.transpose())
adata.obs["labels"] =  meta["labels"].astype("category")

In [ ]:
sc.pp.normalize_total(adata)

In [ ]:
adata.obs = meta

In [ ]:
#sc.tl.rank_genes_groups(adata, 'labels')
cell_groups = meta['labels'].unique().tolist()
matrix_list = {}
for i in cell_groups:
    cells = meta[meta["labels"]==i].index.tolist()
    temp_matrix = matrix[cells]
    matrix_list[i] = (temp_matrix.mean(axis=1)[temp_matrix.mean(axis=1) > 0].index.tolist())

In [ ]:
# adata.obs = meta
# sc.tl.rank_genes_groups(adata, 'labels')

In [ ]:
# sc.tl.rank_genes_groups(adata, 'labels')


# # In[12]:


# pval_df = pd.DataFrame.from_records(adata.uns["rank_genes_groups"]["pvals"])
# pval_df.index = matrix.index.tolist()


# # In[13]:


# cell_type_df = {}
# for i in pval_df.columns.tolist():
#     sub = pval_df[i]
#     sub = sub[sub < 0.05]
#     cell_type_df[i] = sub.index.tolist()


In [ ]:
# adata = adata.transpose()

In [ ]:
#adata.obs["Gene"] = matrix.index.astype("category")

In [ ]:
# pval_df = pd.DataFrame.from_records(adata.uns["rank_genes_groups"]["pvals"])
# pval_df.index = adata.var.index.tolist()

In [ ]:
# cell_type_df = {}
# for i in pval_df.columns.tolist():
#     sub = pval_df[i]
#     sub = sub[sub < 0.05]
#     cell_type_df[i] = sub.index.tolist()
cell_type_df = matrix_list

In [ ]:
#mean_expression = np.mean(matrix.values[matrix.values != 0])
nodes = pd.DataFrame({"category":[],"identifier":[]})

In [ ]:
LR_nodes = pd.read_csv("../data/LR_database/OmniPath_nodes.csv",index_col=0)
Omnipath_network = pd.read_csv("../data/LR_database/OmniPath_interactions.csv",index_col=0)

In [ ]:
ligands = LR_nodes[LR_nodes["category"]=="Ligand"]["identifier"].tolist()
receptors = LR_nodes[LR_nodes["category"]=="Receptor"]["identifier"].tolist()

In [ ]:
ligand_list = []
receptor_list = []
new_cell_df = {}
for i in cell_type_df.keys():
    ligand_list.extend(list(set(ligands) & set(cell_type_df[i])))
    receptor_list.extend(list(set(receptors) & set(cell_type_df[i])))
    new_cell_df[i] = [list(set(ligands) & set(cell_type_df[i])),list(set(receptors) & set(cell_type_df[i]))]

In [ ]:
for i in new_cell_df.keys():
    new_cell_df[i][0] = [j+"_Ligand" for j in new_cell_df[i][0]]
    new_cell_df[i][1] = [j+"_Receptor" for j in new_cell_df[i][1]]

In [ ]:
ligand_list = list(set(ligand_list))
receptor_list = list(set(receptor_list))

In [ ]:
ligand_list = [i+"_Ligand" for i in ligand_list]
receptor_list = [i+"_Receptor" for i in receptor_list]

In [ ]:
nodes = pd.concat([nodes,pd.DataFrame({"category":["Cell Group"] * len(list(cell_type_df.keys())), "identifier":list(cell_type_df.keys())})])
nodes = pd.concat([nodes,pd.DataFrame({"category":["Ligand"] * len(ligand_list), "identifier":ligand_list})])
nodes = pd.concat([nodes,pd.DataFrame({"category":["Receptor"] * len(receptor_list), "identifier":receptor_list})])

In [ ]:
new_identifier = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]

In [ ]:
LR_nodes["identifier"] = new_identifier

In [ ]:
nodes["Id"] = range(0,nodes.shape[0])
nodes = nodes[["Id","category","identifier"]]

In [ ]:
nodes.index = nodes.index.astype('int')
nodes["Id"] = nodes["Id"].astype('int')

In [ ]:
meta.index = meta["cell"]

In [ ]:
interactions = pd.DataFrame({"Src":[],"Dst":[],"Weight":[],"edge_type":[]})

In [ ]:
LR_nodes.index = LR_nodes["Id"].tolist()

In [ ]:
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()


In [ ]:
source_list = []
dest_list = []
weight_list = []
edge_type_list = []
for i in new_cell_df.keys():
    source_list.extend([i] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
    dest_list.extend(new_cell_df[i][0])
    dest_list.extend(new_cell_df[i][1])
    weight_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
    edge_type_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))

In [ ]:
interactions["Src"] = source_list
interactions["Dst"] = dest_list
interactions["Weight"] = weight_list
interactions["edge_type"] = edge_type_list

In [ ]:
nodes.index = nodes["identifier"].tolist()

In [ ]:
nodes = nodes.drop_duplicates("identifier")
nodes["Id"] = range(0,nodes.shape[0])

In [ ]:
interactions["Src"] = nodes.loc[interactions["Src"].tolist()]["Id"].tolist()
interactions["Dst"] = nodes.loc[interactions["Dst"].tolist()]["Id"].tolist()

In [ ]:
first_data,first_nodes,first_interactions = make_dataset(nodes,interactions,first=False,pathway_encode=False)

# Link Prediction with GAT

In [ ]:
from torch_geometric.nn import GATConv
class GAT(torch.nn.Module):
    def __init__(self,data,num_classes=3):
        super(GAT, self).__init__()
        self.hid = 3
        self.in_head = 3
        self.out_head = 3
        
        
        self.conv1 = GATConv(data.x.shape[1], self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head,num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
                
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.logsigmoid(x)
        return x
    
    
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"
    

# Get Omnipath embedding

In [ ]:
LR_nodes = pd.read_csv("../data/LR_database/OmniPath_nodes.csv",index_col=0)
Omnipath_network = pd.read_csv("../data/LR_database/OmniPath_interactions.csv",index_col=0)

In [ ]:
new_identifier = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]

In [ ]:
LR_nodes["identifier"] = new_identifier

In [ ]:
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()


KeyError: "None of [Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,\n            ...\n             693, 1010,  178,   29, 3323,  268,  268,   43,  239,  239],\n           dtype='int64', length=31729)] are in the [index]"

In [ ]:
LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]

In [ ]:
Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]

In [ ]:
LR_nodes["Id"] = range(0,LR_nodes.shape[0])

In [ ]:
LR_nodes.index = LR_nodes["identifier"].tolist()
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()

In [ ]:
LR_nodes = LR_nodes[(LR_nodes["Id"].isin(Omnipath_network["Src"].tolist())) | (LR_nodes["Id"].isin(Omnipath_network["Dst"].tolist()))]

In [ ]:
LR_nodes.index = LR_nodes["Id"].tolist()
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()

In [ ]:
LR_nodes.index = LR_nodes["identifier"].tolist()
LR_nodes["Id"] = range(0,LR_nodes.shape[0])
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()

In [ ]:
from torch_geometric.nn import Node2Vec
import os.path as osp
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm

In [ ]:
def train(model,loader,optimizer):
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in (loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
@torch.no_grad()
def test():
    model.eval()
    z = model()
    acc = model.test(z[data.train_mask], data.y[data.train_mask],
                     z[data.test_mask], data.y[data.test_mask],
                     max_iter=150)
    return acc

In [ ]:
def get_Omnipath_embeddings(nodes,interactions):
    Omnipath_data,Omnipath_nodes,Omnipath_interactions = make_dataset(nodes,interactions,first=False,pathway_encode=False)
    node_info = pd.DataFrame(np.zeros((Omnipath_nodes.shape[0],Omnipath_nodes.shape[0])),index=Omnipath_nodes["identifier"].tolist(),columns=Omnipath_nodes["identifier"].tolist())

    temp_identifiers = [i.split("_")[0] for i in Omnipath_nodes["identifier"].tolist()]

    complexes = pd.read_csv("../data/data/LR_database/complexes.csv")
    complexes = complexes[complexes["member"].isin(temp_identifiers)]

    temp_nodes = Omnipath_nodes.copy()
    temp_nodes.index = temp_identifiers
    temp_nodes = temp_nodes[~temp_nodes.index.duplicated(keep='first')]

    complexes["member"] = temp_nodes.loc[complexes["member"].tolist()]["identifier"].tolist()

    group_complex = complexes.groupby("complex").agg(list)

    group_complex.index=range(0,group_complex.shape[0])

    for index,row in group_complex.iterrows():
        node_info.loc[list(set(row["member"])),list(set(row["member"]))] = index

    # for i in group_complex["member"].tolist():
    #     node_info.loc[list(set(i)),list(set(i))] = 1

    pathways = pd.read_csv("../data/kegg_pathways.csv",index_col=0)
    pathways = pathways[pathways["genesymbol"].isin(temp_identifiers)]
    pathways["genesymbol"] = temp_nodes.loc[pathways["genesymbol"].tolist()]["identifier"].tolist()
    group_pathway = pathways.groupby("pathway").agg(list)

    group_pathway.index=range(0,group_pathway.shape[0])

    for index,row in group_pathway.iterrows():
        node_info.loc[list(set(row["genesymbol"])),list(set(row["genesymbol"]))] += index

    # for i in group_pathway["genesymbol"].tolist():
    #     node_info.loc[list(set(i)),list(set(i))] += 1

    truth_info = pd.DataFrame(np.zeros((Omnipath_nodes.shape[0],Omnipath_nodes.shape[0])),index=Omnipath_nodes["identifier"].tolist(),columns=Omnipath_nodes["identifier"].tolist())

    Omnipath_nodes.index = Omnipath_nodes["Id"].tolist()

    ident_interactions = Omnipath_interactions.copy()
    ident_interactions["Src"] = Omnipath_nodes.loc[ident_interactions["Src"].tolist()]["identifier"].tolist()
    ident_interactions["Dst"] = Omnipath_nodes.loc[ident_interactions["Dst"].tolist()]["identifier"].tolist()

    for index,row in ident_interactions.iterrows():
        truth_info.loc[row["Src"],row["Dst"]] = 1

    ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["identifier"].tolist()
    receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["identifier"].tolist()
    #truth_info = truth_info.loc[ligands,receptors]
    truth_info = torch.Tensor(truth_info.values).to(device)


    ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["Id"].tolist()
    receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["Id"].tolist()


    ident_interactions = ident_interactions.drop_duplicates("Src")
    ident_interactions = ident_interactions.drop_duplicates("Dst")

    ident_interactions.index = range(0,ident_interactions.shape[0])

    truth_list = []
    for i in Omnipath_nodes["identifier"].tolist():
        if "Ligand" in i:
            if i in ident_interactions["Src"].tolist():
                #truth_list.append(ident_interactions[ident_interactions["Src"]==i].index.tolist()[0] + 1)
                truth_list.append(1)
            else:
                truth_list.append(0)
        if "Receptor" in i:
            if i in ident_interactions["Dst"].tolist():
                #truth_list.append(ident_interactions[ident_interactions["Dst"]==i].index.tolist()[0] + 1)
                truth_list.append(1)
            else:
                truth_list.append(0)

    node_info.values[np.where(np.isnan(node_info.values))] = 0
    node_info.values[np.where(np.isinf(node_info.values))] = 0

    Omnipath_data.x = torch.Tensor(node_info.values)

    Omnipath_data.y = torch.Tensor(truth_list).type(torch.LongTensor)
    #Omnipath_data.y = truth_info

    Omnipath_nodes.index = Omnipath_nodes["Id"].tolist()

    Omnipath_interactions["Src"] = [Omnipath_nodes.loc[i]["identifier"] for i in Omnipath_interactions["Src"].tolist()]
    Omnipath_interactions["Dst"] = [Omnipath_nodes.loc[i]["identifier"] for i in Omnipath_interactions["Dst"].tolist()]

    #edge_weights = [max(node_info.loc[i,j],node_info.loc[j,i]) for i,j in zip(Omnipath_interactions["Src"].tolist(),Omnipath_interactions["Dst"].tolist())]
    edge_weights = [1 for i,j in zip(Omnipath_interactions["Src"].tolist(),Omnipath_interactions["Dst"].tolist())]
    data = Omnipath_data
    model = Node2Vec(data.edge_index, embedding_dim=2, walk_length=40,
         context_size=40, walks_per_node=10,
         num_negative_samples=1, p=1, q=1, sparse=True).to(device)

    loader = model.loader(batch_size=2, shuffle=True, num_workers=4)
    optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
    
    
    for epoch in range(1):
        loss = train(model,loader,optimizer)
        #acc = test()
#         if epoch % 10 == 0:
#             print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')
    model.eval()
    z = model(torch.arange(data.num_nodes)).detach()

    ligand_ids = Omnipath_nodes[Omnipath_nodes["category"].str.contains("Ligand")]["Id"].tolist()
    receptor_ids = Omnipath_nodes[Omnipath_nodes["category"].str.contains("Receptor")]["Id"].tolist()

    ligand_embeddings = z[ligand_ids,:]
    receptor_embeddings = z[receptor_ids,:]

    total_embeddings = torch.inner(ligand_embeddings,receptor_embeddings)

    total_embeddings_df = pd.DataFrame(total_embeddings.numpy(),index=Omnipath_nodes[Omnipath_nodes["category"].str.contains("Ligand")]["identifier"].tolist(),columns=Omnipath_nodes[Omnipath_nodes["category"].str.contains("Receptor")]["identifier"].tolist())
    return total_embeddings_df

In [ ]:
    if (Omnipath_network["Src"].dtype != 'float64') and (Omnipath_network["Src"].dtype != 'int64'):
        LR_nodes.index = LR_nodes["identifier"].tolist()
        LR_nodes["Id"] = range(0,LR_nodes.shape[0])
        Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
        Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()
Omnipath_data,Omnipath_nodes,Omnipath_interactions = make_dataset(LR_nodes,Omnipath_network,first=False,pathway_encode=False)
Omnipath_nodes.index = Omnipath_nodes["Id"].tolist()

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
df_list = []
def get_cell_LR_embeddings(nodes,interactions,total_embeddings_df,Omnipath_nodes):
    device = 'cpu'
    truth_info = pd.DataFrame(np.zeros((Omnipath_nodes.shape[0],Omnipath_nodes.shape[0])),index=Omnipath_nodes["identifier"].tolist(),columns=Omnipath_nodes["identifier"].tolist())

    #Omnipath_nodes.index = Omnipath_nodes["Id"].tolist()

    ident_interactions = Omnipath_interactions.copy()
    ident_interactions["Src"] = Omnipath_nodes.loc[ident_interactions["Src"].tolist()]["identifier"].tolist()
    ident_interactions["Dst"] = Omnipath_nodes.loc[ident_interactions["Dst"].tolist()]["identifier"].tolist()

    for index,row in ident_interactions.iterrows():
        truth_info.loc[row["Src"],row["Dst"]] = 1

    ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["identifier"].tolist()
    receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["identifier"].tolist()
    #truth_info = truth_info.loc[ligands,receptors]
    truth_info = torch.Tensor(truth_info.values).to(device)

    
    cell_LR_data,cell_LR_nodes,cell_LR_ints = make_dataset(nodes,interactions,first=False,pathway_encode=False)

    full_matrix = pd.DataFrame(np.zeros((cell_LR_nodes.shape[0],cell_LR_nodes.shape[0])),index=cell_LR_nodes["identifier"].tolist(),columns=cell_LR_nodes["identifier"].tolist())

    ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["Id"].tolist()
    receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["Id"].tolist()

    total_out_df = total_embeddings_df

    gene_mean = matrix.mean(axis=1)

    Omnipath_nodes.index = [i.split("_")[0] for i in Omnipath_nodes["identifier"].tolist()]

    Omnipath_nodes = Omnipath_nodes.loc[~Omnipath_nodes.index.duplicated(),:].copy()
    

    gene_mean = gene_mean.loc[Omnipath_nodes.index.tolist()]

    gene_mean.index = Omnipath_nodes["identifier"].tolist()

    ligands = [i for i in gene_mean.index.tolist() if "Ligand" in i]
    receptors = [i for i in gene_mean.index.tolist() if "Receptor" in i]
    ligands = list(set(full_matrix.index.tolist()) & set(ligands))
    receptors = list(set(full_matrix.index.tolist()) & set(receptors))

    for i,j in zip(ligands,receptors):
        if (i in gene_mean.index.tolist()) and (j in gene_mean.index.tolist()):
            full_matrix.loc[i,j] = gene_mean.loc[i]*gene_mean.loc[j]

    ligands = list(set(ligands) & set(total_out_df.index.tolist()))
    receptors = list(set(receptors) & set(total_out_df.columns.tolist()))

    for i,j in zip(ligands,receptors):
        full_matrix.loc[i,j] += total_out_df.loc[i,j]


    #full_matrix.loc[true_df.index.tolist(),true_df.columns.tolist()] = true_df.values

    cell_groups = meta["labels"].unique().tolist()

    ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["identifier"].tolist()
    ligands = [i.split("_")[0] for i in ligands]
    ligand_matrix = matrix.loc[ligands]

    ligand_matrix = ligand_matrix[~ligand_matrix.index.duplicated(keep='first')]


    mean_dict = {}
    for i in cell_groups:
        cells = meta[meta["labels"]==i]["cell"].tolist()
        mean_dict[i] = ligand_matrix[cells].mean(axis=1)

    full_matrix = full_matrix[~full_matrix.index.duplicated(keep='first')]
    full_matrix = full_matrix.loc[:,~full_matrix.columns.duplicated()].copy()


    for i in mean_dict.keys():
        temp_index = [i+"_Ligand" for i in mean_dict[i].index.tolist()]
        full_matrix.loc[i,temp_index] = mean_dict[i].values
        full_matrix.loc[temp_index,i] = mean_dict[i].values

    receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["identifier"].tolist()
    receptors = [i.split("_")[0] for i in receptors]
    receptors_matrix = matrix.loc[receptors]

    receptors_matrix = receptors_matrix[~receptors_matrix.index.duplicated(keep='first')]


    mean_dict = {}
    for i in cell_groups:
        cells = meta[meta["labels"]==i]["cell"].tolist()
        mean_dict[i] = receptors_matrix[cells].mean(axis=1)

    full_matrix = full_matrix[~full_matrix.index.duplicated(keep='first')]
    full_matrix = full_matrix.loc[:,~full_matrix.columns.duplicated()].copy()


    for i in mean_dict.keys():
        temp_index = [i+"_Receptor" for i in mean_dict[i].index.tolist()]
        full_matrix.loc[i,temp_index] = mean_dict[i].values
        full_matrix.loc[temp_index,i] = mean_dict[i].values

    #full_matrix = (full_matrix - np.min(full_matrix)) / (np.max(full_matrix) - np.min(full_matrix))

    full_matrix.values[np.where(np.isnan(full_matrix.values))] = 0
    full_matrix.values[np.where(np.isinf(full_matrix.values))] = 0

    adata = sc.read_h5ad("../data/Sikander_data/Visium-FZ_GT_P19.h5ad")
    adata = adata[meta.index.tolist()]

    spatial_coordinates = adata.obsm["X_spatial"]

    spatial_df = pd.DataFrame({"x":spatial_coordinates[:,0],"y":spatial_coordinates[:,1]},index=meta.index.tolist())

    cell_groups = meta["labels"].unique().tolist()

    import math

    spatial_dict = {}
    for i in cell_groups:
        cells = meta[meta["labels"]==i].index.tolist()
        spatial_coords = [(i,j) for i,j in zip(spatial_df.loc[cells]["x"].tolist(),spatial_df.loc[cells]["y"].tolist())]
        for j in cell_groups:
            if j != i:
                second_cells = meta[meta["labels"]==j].index.tolist()
                second_spatial_coords = [(i,j) for i,j in zip(spatial_df.loc[second_cells]["x"].tolist(),spatial_df.loc[second_cells]["y"].tolist())]
                min_list = []
                for k in spatial_coords:
                    min_list.append(min([math.dist(k,l) for l in second_spatial_coords]))
                spatial_dict[(i,j)] = min(min_list)


    for k in spatial_dict.keys():
        full_matrix.loc[k[0],k[1]] = spatial_dict[k]
        full_matrix.loc[k[1],k[0]] = spatial_dict[k]

    cell_LR_data.x = torch.Tensor(full_matrix.values)

    LR_ids = cell_LR_nodes[(cell_LR_nodes["category"]=="Ligand") | (cell_LR_nodes["category"]=="Receptor")]["Id"].tolist()

    # true_values["Src"] = [i + "_Ligand" for i in true_values["Src"].tolist()]
    # true_values["Dst"] = [i + "_Receptor" for i in true_values["Dst"].tolist()]

    cell_groups = cell_LR_nodes[cell_LR_nodes["category"]=="Cell Group"]['identifier'].tolist()

    truth_list = []
    for i in cell_LR_nodes["identifier"].tolist():
        if "Ligand" in i:
            if i in ident_interactions["Src"].tolist():
                #truth_list.append(ident_interactions[ident_interactions["Src"]==i].index.tolist()[0] + 1)
                truth_list.append(1)
            else:
                truth_list.append(0)
        elif "Receptor" in i:
            if i in ident_interactions["Dst"].tolist():
                #truth_list.append(ident_interactions[ident_interactions["Dst"]==i].index.tolist()[0] + 1)
                truth_list.append(1)
            else:
                truth_list.append(0)
        else:
            truth_list.append(2)

    cell_LR_data.y = torch.Tensor(truth_list).type(torch.LongTensor)

    truth_array = np.array(truth_list)
    positive_classes = np.where(truth_array==1)[0].tolist()
    negative_classes = np.where(truth_array==0)[0].tolist()[:len(positive_classes)]

    new_train_mask = np.array([False]*truth_array.shape[0])
    new_train_mask[positive_classes + negative_classes] = True

    model = GAT(cell_LR_data,num_classes=2).to(device)
    data = cell_LR_data.to(device)
    data.train_mask = torch.Tensor(new_train_mask).type(torch.LongTensor)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()

    ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["Id"].tolist()
    receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["Id"].tolist()

    truth_df = full_matrix.loc[cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["identifier"].tolist(),cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["identifier"].tolist()]

    truth_Tensor = torch.Tensor(truth_df.values).to(device)

    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        out = model(data)
        #loss = F.nll_loss(out[new_train_mask],data.y[new_train_mask])
        ligand_out = out[ligands,:]
        receptor_out = out[receptors,:]
        total_out = torch.inner(ligand_out,receptor_out)
        #loss = criterion(out[LR_ids],data.y)
        #loss = criterion(total_out,truth_Tensor)
        loss = criterion(out[new_train_mask],data.y[new_train_mask])
        loss.backward()
        optimizer.step()


    model.eval()
    cell_LR_out = model(data)

    ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["Id"].tolist()
    receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["Id"].tolist()

    ligand_out = cell_LR_out[ligands,:]
    receptor_out = cell_LR_out[receptors,:]
    _,ligand_pred = ligand_out.max(dim=1)
    _,receptor_pred = receptor_out.max(dim=1)

    total_out = torch.inner(ligand_out,receptor_out).cpu().detach().numpy()
    ligand_pred = ligand_pred.cpu().detach().numpy()
    receptor_pred = receptor_pred.cpu().detach().numpy()
    
    cell_LR_nodes.index = cell_LR_nodes["Id"].tolist()
    valid_ligands = cell_LR_nodes.loc[np.where(ligand_pred == 1)]
    valid_receptors = cell_LR_nodes.loc[np.where(receptor_pred == 1)]
    ligand_out = ligand_out[np.where(ligand_pred ==1)]
    receptor_out = receptor_out[np.where(receptor_pred ==1)]  
    total_out = torch.inner(ligand_out,receptor_out).cpu().detach().numpy()
    ligand_nodes = cell_LR_nodes[cell_LR_nodes["category"] == "Ligand"]
    ligand_nodes.index = range(0,ligand_nodes.shape[0])
    ligand_idents = ligand_nodes.iloc[np.where(ligand_pred==1)]['identifier'].tolist()
    total_out_df = pd.DataFrame(total_out,index=valid_ligands["identifier"].tolist(),columns=valid_receptors["identifier"].tolist())
    indicies = np.where(total_out_df.values > 0)
    source = list(indicies[0])
    dest = list(indicies[1])
    index_df = pd.DataFrame({"Id":range(0,total_out_df.shape[0]),"identifier":total_out_df.index.tolist()})
    column_df = pd.DataFrame({"Id":range(0,total_out_df.shape[1]),"identifier":total_out_df.columns.tolist()})
    source_list = index_df.loc[source]["identifier"].tolist()
    dest_list = column_df.loc[dest]["identifier"].tolist()
    total_link_df = pd.DataFrame({"Src":source_list,"Dst":dest_list,"Prob":total_out_df.values[indicies]})
    total_link_df = total_link_df.sort_values("Prob",ascending=False)
    Omnipath_db = pd.read_csv("../data/LR_database/Omnipath_database.csv")
    total_link_df["Src"] = [i.split("_")[0] for i in total_link_df["Src"].tolist()]
    total_link_df["Dst"] = [i.split("_")[0] for i in total_link_df["Dst"].tolist()]
    total_link_df = total_link_df.drop_duplicates()
    return total_link_df

In [117]:
df_list = []
for i in range(100):
    print(i)
    Omnipath_network["Src"] = Omnipath_network["Src"].sample(frac=1).tolist()
    Omnipath_network["Dst"] = Omnipath_network["Dst"].sample(frac=1).tolist()
    if (Omnipath_network["Src"].dtype != 'float64') and (Omnipath_network["Src"].dtype != 'int64'):
        LR_nodes.index = LR_nodes["identifier"].tolist()
        LR_nodes["Id"] = range(0,LR_nodes.shape[0])
        Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
        Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()
    print("Done Preprocessing")
    df = get_Omnipath_embeddings(LR_nodes,Omnipath_network)
    print("Done Omnipath embeddings")
    interactions["Src"] = interactions["Src"].sample(frac=1).tolist()
    interactions["Dst"] = interactions["Dst"].sample(frac=1).tolist()  
    Omnipath_nodes.index = Omnipath_nodes["identifier"].tolist()
    df_list.append(get_cell_LR_embeddings(nodes,interactions,df,Omnipath_nodes))
    print("Done cell embeddings")

0
Done Preprocessing
Done Omnipath embeddings



KeyboardInterrupt



In [118]:
Omnipath_network["Src"] = Omnipath_network["Src"].sample(frac=1).tolist()
Omnipath_network["Dst"] = Omnipath_network["Dst"].sample(frac=1).tolist()
if (Omnipath_network["Src"].dtype != 'float64') and (Omnipath_network["Src"].dtype != 'int64'):
    LR_nodes.index = LR_nodes["identifier"].tolist()
    LR_nodes["Id"] = range(0,LR_nodes.shape[0])
    Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
    Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()
print("Done Preprocessing")
df = get_Omnipath_embeddings(LR_nodes,Omnipath_network)

Done Preprocessing


In [119]:
    interactions["Src"] = interactions["Src"].sample(frac=1).tolist()
    interactions["Dst"] = interactions["Dst"].sample(frac=1).tolist()  
    Omnipath_nodes.index = Omnipath_nodes["identifier"].tolist()

In [120]:
total_embeddings_df = df

In [121]:
Omnipath_nodes

,Id,category,identifier
NRP2_Ligand,0,Ligand,NRP2_Ligand
NGFR_Ligand,1,Ligand,NGFR_Ligand
ENG_Ligand,2,Ligand,ENG_Ligand
RYK_Ligand,3,Ligand,RYK_Ligand
GAS1_Ligand,4,Ligand,GAS1_Ligand
...,...,...,...
PPP2R1B_Receptor,6395,Receptor,PPP2R1B_Receptor
LMO4_Receptor,6396,Receptor,LMO4_Receptor
HEY1_Receptor,6397,Receptor,HEY1_Receptor
ZNF107_Receptor,6398,Receptor,ZNF107_Receptor


In [122]:
ident_interactions

,Src,Dst,Weight,edge_type
0,SH3GLB1_Ligand,PTH1R_Receptor,1,0
1,FGR_Ligand,WIPF1_Receptor,1,0
2,HLA-C_Ligand,PPP4C_Receptor,1,0
3,IL1RAP_Ligand,PDPN_Receptor,1,0
4,KDM6A_Ligand,LILRA6_Receptor,1,0
...,...,...,...,...
40461,KLF6_Ligand,FGR_Receptor,1,0
40463,CYLD_Ligand,PTGER2_Receptor,1,0
40464,EHMT2_Ligand,MAP2K4_Receptor,1,0
40465,RARRES1_Ligand,MUC1_Receptor,1,0


In [123]:
device = 'cpu'
truth_info = pd.DataFrame(np.zeros((Omnipath_nodes.shape[0],Omnipath_nodes.shape[0])),index=Omnipath_nodes["identifier"].tolist(),columns=Omnipath_nodes["identifier"].tolist())

#Omnipath_nodes.index = Omnipath_nodes["Id"].tolist()

ident_interactions = Omnipath_interactions.copy()
ident_interactions["Src"] = Omnipath_nodes.loc[ident_interactions["Src"].tolist()]["identifier"].tolist()
ident_interactions["Dst"] = Omnipath_nodes.loc[ident_interactions["Dst"].tolist()]["identifier"].tolist()

for index,row in ident_interactions.iterrows():
    truth_info.loc[row["Src"],row["Dst"]] = 1

ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["identifier"].tolist()
receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["identifier"].tolist()
#truth_info = truth_info.loc[ligands,receptors]
truth_info = torch.Tensor(truth_info.values).to(device)


cell_LR_data,cell_LR_nodes,cell_LR_ints = make_dataset(nodes,interactions,first=False,pathway_encode=False)

full_matrix = pd.DataFrame(np.zeros((cell_LR_nodes.shape[0],cell_LR_nodes.shape[0])),index=cell_LR_nodes["identifier"].tolist(),columns=cell_LR_nodes["identifier"].tolist())

ligands = Omnipath_nodes[Omnipath_nodes["category"]=="Ligand"]["Id"].tolist()
receptors = Omnipath_nodes[Omnipath_nodes["category"]=="Receptor"]["Id"].tolist()

total_out_df = total_embeddings_df

gene_mean = matrix.mean(axis=1)

Omnipath_nodes.index = [i.split("_")[0] for i in Omnipath_nodes["identifier"].tolist()]

Omnipath_nodes = Omnipath_nodes.loc[~Omnipath_nodes.index.duplicated(),:].copy()


gene_mean = gene_mean.loc[Omnipath_nodes.index.tolist()]

gene_mean.index = Omnipath_nodes["identifier"].tolist()

ligands = [i for i in gene_mean.index.tolist() if "Ligand" in i]
receptors = [i for i in gene_mean.index.tolist() if "Receptor" in i]
ligands = list(set(full_matrix.index.tolist()) & set(ligands))
receptors = list(set(full_matrix.index.tolist()) & set(receptors))

for i,j in zip(ligands,receptors):
    if (i in gene_mean.index.tolist()) and (j in gene_mean.index.tolist()):
        full_matrix.loc[i,j] = gene_mean.loc[i]*gene_mean.loc[j]

ligands = list(set(ligands) & set(total_out_df.index.tolist()))
receptors = list(set(receptors) & set(total_out_df.columns.tolist()))

for i,j in zip(ligands,receptors):
    full_matrix.loc[i,j] += total_out_df.loc[i,j]


#full_matrix.loc[true_df.index.tolist(),true_df.columns.tolist()] = true_df.values

cell_groups = meta["labels"].unique().tolist()

ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["identifier"].tolist()
ligands = [i.split("_")[0] for i in ligands]
ligand_matrix = matrix.loc[ligands]

ligand_matrix = ligand_matrix[~ligand_matrix.index.duplicated(keep='first')]


mean_dict = {}
for i in cell_groups:
    cells = meta[meta["labels"]==i]["cell"].tolist()
    mean_dict[i] = ligand_matrix[cells].mean(axis=1)

full_matrix = full_matrix[~full_matrix.index.duplicated(keep='first')]
full_matrix = full_matrix.loc[:,~full_matrix.columns.duplicated()].copy()


for i in mean_dict.keys():
    temp_index = [i+"_Ligand" for i in mean_dict[i].index.tolist()]
    full_matrix.loc[i,temp_index] = mean_dict[i].values
    full_matrix.loc[temp_index,i] = mean_dict[i].values

receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["identifier"].tolist()
receptors = [i.split("_")[0] for i in receptors]
receptors_matrix = matrix.loc[receptors]

receptors_matrix = receptors_matrix[~receptors_matrix.index.duplicated(keep='first')]


mean_dict = {}
for i in cell_groups:
    cells = meta[meta["labels"]==i]["cell"].tolist()
    mean_dict[i] = receptors_matrix[cells].mean(axis=1)

full_matrix = full_matrix[~full_matrix.index.duplicated(keep='first')]
full_matrix = full_matrix.loc[:,~full_matrix.columns.duplicated()].copy()


for i in mean_dict.keys():
    temp_index = [i+"_Receptor" for i in mean_dict[i].index.tolist()]
    full_matrix.loc[i,temp_index] = mean_dict[i].values
    full_matrix.loc[temp_index,i] = mean_dict[i].values

#full_matrix = (full_matrix - np.min(full_matrix)) / (np.max(full_matrix) - np.min(full_matrix))

full_matrix.values[np.where(np.isnan(full_matrix.values))] = 0
full_matrix.values[np.where(np.isinf(full_matrix.values))] = 0

adata = sc.read_h5ad("../data/Sikander_data/Visium-FZ_GT_P19.h5ad")
adata = adata[meta.index.tolist()]

spatial_coordinates = adata.obsm["X_spatial"]

spatial_df = pd.DataFrame({"x":spatial_coordinates[:,0],"y":spatial_coordinates[:,1]},index=meta.index.tolist())

cell_groups = meta["labels"].unique().tolist()

import math

spatial_dict = {}
for i in cell_groups:
    cells = meta[meta["labels"]==i].index.tolist()
    spatial_coords = [(i,j) for i,j in zip(spatial_df.loc[cells]["x"].tolist(),spatial_df.loc[cells]["y"].tolist())]
    for j in cell_groups:
        if j != i:
            second_cells = meta[meta["labels"]==j].index.tolist()
            second_spatial_coords = [(i,j) for i,j in zip(spatial_df.loc[second_cells]["x"].tolist(),spatial_df.loc[second_cells]["y"].tolist())]
            min_list = []
            for k in spatial_coords:
                min_list.append(min([math.dist(k,l) for l in second_spatial_coords]))
            spatial_dict[(i,j)] = min(min_list)


for k in spatial_dict.keys():
    full_matrix.loc[k[0],k[1]] = spatial_dict[k]
    full_matrix.loc[k[1],k[0]] = spatial_dict[k]

cell_LR_data.x = torch.Tensor(full_matrix.values)

LR_ids = cell_LR_nodes[(cell_LR_nodes["category"]=="Ligand") | (cell_LR_nodes["category"]=="Receptor")]["Id"].tolist()

# true_values["Src"] = [i + "_Ligand" for i in true_values["Src"].tolist()]
# true_values["Dst"] = [i + "_Receptor" for i in true_values["Dst"].tolist()]

cell_groups = cell_LR_nodes[cell_LR_nodes["category"]=="Cell Group"]['identifier'].tolist()

truth_list = []
for i in cell_LR_nodes["identifier"].tolist():
    if "Ligand" in i:
        if i in ident_interactions["Src"].tolist():
            #truth_list.append(ident_interactions[ident_interactions["Src"]==i].index.tolist()[0] + 1)
            truth_list.append(1)
        else:
            truth_list.append(0)
    elif "Receptor" in i:
        if i in ident_interactions["Dst"].tolist():
            #truth_list.append(ident_interactions[ident_interactions["Dst"]==i].index.tolist()[0] + 1)
            truth_list.append(1)
        else:
            truth_list.append(0)
    else:
        truth_list.append(2)

cell_LR_data.y = torch.Tensor(truth_list).type(torch.LongTensor)

truth_array = np.array(truth_list)
positive_classes = np.where(truth_array==1)[0].tolist()
negative_classes = np.where(truth_array==0)[0].tolist()[:len(positive_classes)]

new_train_mask = np.array([False]*truth_array.shape[0])
new_train_mask[positive_classes + negative_classes] = True

model = GAT(cell_LR_data,num_classes=2).to(device)
data = cell_LR_data.to(device)
data.train_mask = torch.Tensor(new_train_mask).type(torch.LongTensor)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["Id"].tolist()
receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["Id"].tolist()

truth_df = full_matrix.loc[cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["identifier"].tolist(),cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["identifier"].tolist()]

truth_Tensor = torch.Tensor(truth_df.values).to(device)

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    #loss = F.nll_loss(out[new_train_mask],data.y[new_train_mask])
    ligand_out = out[ligands,:]
    receptor_out = out[receptors,:]
    total_out = torch.inner(ligand_out,receptor_out)
    #loss = criterion(out[LR_ids],data.y)
    #loss = criterion(total_out,truth_Tensor)
    loss = criterion(out[new_train_mask],data.y[new_train_mask])
    loss.backward()
    optimizer.step()


model.eval()
cell_LR_out = model(data)

ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["Id"].tolist()
receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["Id"].tolist()

ligand_out = cell_LR_out[ligands,:]
receptor_out = cell_LR_out[receptors,:]
_,ligand_pred = ligand_out.max(dim=1)
_,receptor_pred = receptor_out.max(dim=1)

total_out = torch.inner(ligand_out,receptor_out).cpu().detach().numpy()
ligand_pred = ligand_pred.cpu().detach().numpy()
receptor_pred = receptor_pred.cpu().detach().numpy()

cell_LR_nodes.index = cell_LR_nodes["Id"].tolist()
valid_ligand_df = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]
valid_ligand_df.index = range(valid_ligand_df.shape[0])
valid_receptor_df = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]
valid_receptor_df.index = range(valid_receptor_df.shape[0])

valid_ligands = valid_ligand_df.loc[np.where(ligand_pred == 1)]
valid_receptors = valid_receptor_df.loc[np.where(receptor_pred == 1)]
ligand_out = ligand_out[np.where(ligand_pred ==1)]
receptor_out = receptor_out[np.where(receptor_pred ==1)]  
total_out = torch.inner(ligand_out,receptor_out).cpu().detach().numpy()
ligand_nodes = cell_LR_nodes[cell_LR_nodes["category"] == "Ligand"]
ligand_nodes.index = range(0,ligand_nodes.shape[0])
ligand_idents = ligand_nodes.iloc[np.where(ligand_pred==1)]['identifier'].tolist()
total_out_df = pd.DataFrame(total_out,index=valid_ligands["identifier"].tolist(),columns=valid_receptors["identifier"].tolist())
indicies = np.where(total_out_df.values > 0)
source = list(indicies[0])
dest = list(indicies[1])
index_df = pd.DataFrame({"Id":range(0,total_out_df.shape[0]),"identifier":total_out_df.index.tolist()})
column_df = pd.DataFrame({"Id":range(0,total_out_df.shape[1]),"identifier":total_out_df.columns.tolist()})
source_list = index_df.loc[source]["identifier"].tolist()
dest_list = column_df.loc[dest]["identifier"].tolist()
total_link_df = pd.DataFrame({"Src":source_list,"Dst":dest_list,"Prob":total_out_df.values[indicies]})
total_link_df = total_link_df.sort_values("Prob",ascending=False)
Omnipath_db = pd.read_csv("../data/LR_database/Omnipath_database.csv")
total_link_df["Src"] = [i.split("_")[0] for i in total_link_df["Src"].tolist()]
total_link_df["Dst"] = [i.split("_")[0] for i in total_link_df["Dst"].tolist()]
total_link_df = total_link_df.drop_duplicates()

In [125]:
    LR_out = cell_LR_out[valid_ligands["Id"].tolist() + valid_receptors["Id"].tolist(),:]
    cell_group_out = cell_LR_out[cell_LR_nodes[cell_LR_nodes["category"]=="Cell Group"]["Id"].tolist(),:]
    cell_LR_out = torch.inner(LR_out,cell_group_out).cpu().detach().numpy()

    cell_LR_df = pd.DataFrame(cell_LR_out,index=valid_ligands["identifier"].tolist() + valid_receptors["identifier"].tolist(),columns=cell_LR_nodes[cell_LR_nodes["category"]=="Cell Group"]["identifier"].tolist())

    ligands = nodes[nodes["category"]=="Ligand"]["identifier"].tolist()
    receptors = nodes[nodes["category"]=="Receptor"]["identifier"].tolist()
    expression_df = matrix

    cell_groups = meta["labels"].unique().tolist()


    # In[30]:


    mean_matrix = pd.DataFrame(columns=cell_groups,index=ligands+receptors)


    # In[31]:


    for i in cell_groups:
        cells = meta[meta["labels"]==i]["cell"].tolist()
        temp_ligands = [i.split("_")[0] for i in ligands]
        ligand_df = expression_df[cells].mean(axis=1).loc[temp_ligands]
        ligand_df.index = [i+"_Ligand" for i in ligand_df.index.tolist()]
        temp_receptors = receptors = [i.split("_")[0] for i in receptors]
        receptor_df = expression_df[cells].mean(axis=1).loc[temp_receptors]
        receptor_df.index = [i+"_Receptor" for i in receptor_df.index.tolist()]
        total_df = pd.concat([ligand_df,receptor_df])
        mean_matrix[i] = total_df.tolist()

    ligands = cell_LR_nodes[cell_LR_nodes["category"]=="Ligand"]["Id"].tolist()
    receptors = cell_LR_nodes[cell_LR_nodes["category"]=="Receptor"]["Id"].tolist()

    interacting_ligands = list(set(ligands) & set(cell_LR_ints["Dst"].tolist()))
    interacting_receptors = list(set(receptors) & set(cell_LR_ints["Dst"].tolist()))


    # In[99]:


    cell_LR_ints.index = cell_LR_ints["Dst"].tolist()
    ligand_cells = cell_LR_ints.loc[interacting_ligands]["Src"].unique().tolist()
    receptor_cells = cell_LR_ints.loc[interacting_receptors]["Src"].unique().tolist()

    cell_LR_out = torch.Tensor(cell_LR_out)

    ligand_cell_out = out[ligand_cells,:]
    ligand_out = out[ligands,:]
    total_ligand_out = torch.inner(ligand_out,ligand_cell_out).cpu().detach().numpy()
    receptor_cell_out = out[receptor_cells,:]
    receptor_out = out[receptors,:]
    total_receptor_out = torch.inner(receptor_out,receptor_cell_out).cpu().detach().numpy()

    ligand_matrix = mean_matrix[mean_matrix.index.str.contains("Ligand")]
    receptor_matrix = mean_matrix[mean_matrix.index.str.contains("Receptor")]


    # In[112]:


    ligand_cell_out = np.multiply(ligand_matrix,total_ligand_out)
    receptor_cell_out = np.multiply(receptor_matrix,total_receptor_out)


    # In[113]:


    ligand_maxes = (ligand_cell_out.idxmax(axis=1))
    receptor_maxes = (receptor_cell_out.idxmax(axis=1))


    ligand_maxes.index = [i.split("_")[0] for i in ligand_maxes.index.tolist()]
    receptor_maxes.index = [i.split("_")[0] for i in receptor_maxes.index.tolist()]

    total_link_df["Src Cell"] = ligand_maxes.loc[total_link_df['Src']].tolist()
    total_link_df["Dst Cell"] = receptor_maxes.loc[total_link_df['Dst']].tolist()

In [126]:
total_link_df

,Src,Dst,Prob,Src Cell,Dst Cell
13230108,ACAN,KCNJ5,1.727050,vSMCs,Myeloid
19216248,MED1,KCNJ5,1.726681,Myeloid,Myeloid
19150299,CLEC11A,KCNJ5,1.726379,vSMCs,Myeloid
15827484,UBASH3B,KCNJ5,1.726071,Myeloid,Myeloid
13226656,ACAN,GYS1,1.725974,vSMCs,Pericyte
...,...,...,...,...,...
12748060,ALPL,PPP1R1B,1.685342,Endothelial,Fibroblast
8218906,SLAMF6,HOXB7,1.684533,Myeloid,Myeloid
12745564,ALPL,ENAM,1.683755,Endothelial,Cardiomyocyte
8222944,SLAMF6,PPP1R1B,1.682235,Myeloid,Fibroblast
